In [41]:
# The basics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Spark library and functions
import pyspark as ps
import pyspark.sql.types as types
from pyspark.sql.functions import col, countDistinct
from pyspark.sql.functions import to_timestamp

# Sklearn Modeling
from sklearn.model_selection import train_test_split


# Surprise modeling
from surprise import SVD
import surprise
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate

# Persistance
import pickle

# Housekeeping
from io import StringIO

In [5]:
# review_df = pd.read_json('review.json')

In [11]:
spark = (ps.sql.SparkSession
         .builder
         .master('local[4]')
         .appName('lecture')
         .getOrCreate()
        )
# sc = spark.sparkContext

review_df = spark.read.json("data/review.json")
# checkin_df = spark.read.json("data/checkin.json")
# tip_df = spark.read.json("data/tip.json")
# user_df = spark.read.json("data/user.json")
business_df = spark.read.json("data/business.json")

In [5]:
# Displays the content of the DataFrame to stdout
# review_df.columns

In [6]:
# checkin_df.head()

In [7]:
# tip_df.head()

In [8]:
# photo_df.head()

In [4]:
business_df.take(2)

[Row(address='2818 E Camino Acequia Drive', attributes=Row(AcceptsInsurance=None, AgesAllowed=None, Alcohol=None, Ambience=None, BYOB=None, BYOBCorkage=None, BestNights=None, BikeParking=None, BusinessAcceptsBitcoin=None, BusinessAcceptsCreditCards=None, BusinessParking=None, ByAppointmentOnly=None, Caters=None, CoatCheck=None, Corkage=None, DietaryRestrictions=None, DogsAllowed=None, DriveThru=None, GoodForDancing=None, GoodForKids='False', GoodForMeal=None, HairSpecializesIn=None, HappyHour=None, HasTV=None, Music=None, NoiseLevel=None, Open24Hours=None, OutdoorSeating=None, RestaurantsAttire=None, RestaurantsCounterService=None, RestaurantsDelivery=None, RestaurantsGoodForGroups=None, RestaurantsPriceRange2=None, RestaurantsReservations=None, RestaurantsTableService=None, RestaurantsTakeOut=None, Smoking=None, WheelchairAccessible=None, WiFi=None), business_id='1SWheh84yJXfytovILXOAQ', categories='Golf, Active Life', city='Phoenix', hours=None, is_open=0, latitude=33.5221425, longit

In [10]:
# user_df.columns

# Basic Cleaning

In [ ]:
# # Check for null values
# frames = [business_df, review_df, user_df]

# for df in frames:
#     for c in df.columns:
#         null_value_count = df.where(df[c] == None).count()
#         print('Column: {}\nNull count: {}\n\n'.format(c, str(null_value_count)))
    
# # This is not showing any null values for the moment, remain skeptical, possible misuse of spark

# Basic EDA

In [12]:
# Count of distinct values for all fields in review_df:
this = business_df.agg(*(countDistinct(col(c)).alias(c) for c in business_df.columns))
this.show(1)

+-------+----------+-----------+----------+----+-----+-------+--------+---------+------+-----------+------------+-----+-----+
|address|attributes|business_id|categories|city|hours|is_open|latitude|longitude|  name|postal_code|review_count|stars|state|
+-------+----------+-----------+----------+----+-----+-------+--------+---------+------+-----------+------------+-----+-----+
| 151977|     67875|     192609|     93385|1204|51566|      2|  155162|   150404|145046|      17541|        1184|    9|   36|
+-------+----------+-----------+----------+----+-----+-------+--------+---------+------+-----------+------------+-----+-----+



In [14]:
this = review_df.agg(*(countDistinct(col(c)).alias(c) for c in review_df.columns))
this.show(1)

+-----------+----+-------+-----+---------+-----+-------+------+-------+
|business_id|cool|   date|funny|review_id|stars|   text|useful|user_id|
+-----------+----+-------+-----+---------+-----+-------+------+-------+
|     192606| 201|6552820|  231|  6685900|    5|6668738|   289|1637138|
+-----------+----+-------+-----+---------+-----+-------+------+-------+



In [ ]:
bsubset_df = business_df.filter((business_df.categories.like('%Restaurants%')) &
                                (business_df.state == 'AZ')                               
                               )
print('There are {} Arizona restaurants in the dataset.'.format(bsubset_df.count()))

## Major Metros in Dataset
#### Las Vegas
* Henderson
#### Toronto
* Missassauga
* Markham
* North York
#### Phoenix - 56,686
* Mesa
* Tempe
* Scottsdale
* Chandler
* Glendale
* Gilbert
* Peoria
* Surprise
#### Calgary
#### Pittsburgh
#### Montreal
#### Cleveland
#### Madison, WI
#### Champaign, IL

### Which region should we select for proceeding with this study? 

Let's look at the densities to decide:

In [ ]:
def calculate_region_density(state, businesses, bus_reviews):
    num_businesses = businesses.filter((businesses.categories.like('%Restaurants%')) &
                                (business_df.state == state)).count()
    num_reviews = bus_reviews.filter((bus_reviews.categories.like('%Restaurants%')) &
                                (bus_reviews.state == state)).count()
    return num_reviews/num_businesses

def compare_region_densities(regions, businesses, bus_reviews):
    max_density = 0
    density_dict = {}
    for region in regions:
        density = calculate_region_density(region, businesses, bus_reviews)
        density_dict[region] = density
        if density > max_density:
            max_density = density
            max_region = region

    print('Best Region: {}\nBest Density: {}'.format(max_region, max_density))
    return density_dict

In [ ]:
regions = ['AZ', 'ON', 'NC', 'AB', 'NV', 'OH', 'PA', 'IL', 'SC', 'QC']

density_dict = compare_region_densities(regions, business_df, bus_review_df)

pd.DataFrame.from_dict(density_dict, orient = 'index').sort_values(0, ascending = False).plot.bar(legend = False, figsize = (10, 6))
plt.ylabel('Review Density')
plt.xlabel('Region')
plt.title('Review Density by Region')

## The winner: Arizona

We are proceeding with Arizona based businesses for this case study. Although Nevada has a greater density of reviews/restaurant, this data scientist considers it to be an anomaly in the restaurant space as well as user space given the impact tourism would have on restaurant visits/reviews. Arizona has the next highest density, would be expected to include a healthy mix of resident and non-resident reviews, most reviews are concentrated around Phoenix, and we would expect Phoenix to have a diverse group of users.

# Query for reviews of restaurants in Arizona and then convert to Pandas

In [11]:
# Make "star" columns unique on business_df and review_df to avoid confusion
business_df = business_df.withColumnRenamed("stars","avg_stars")
review_df = review_df.withColumnRenamed("stars","review_stars")

In [12]:
# Filter business_df down to only businesses in Arizona
business_df = business_df.filter((business_df.state == 'AZ') &
                                (business_df.categories.like('%Restaurants%'))
                                )

In [13]:
# Join business and review df's 
bus_review_df = review_df.join(business_df, review_df.business_id == business_df.business_id, how='left')

In [1]:
# # Convert to pandas to make it easier to work
# bus_review_df = bus_review_df.select("*").toPandas()

# # Not Currently working on my machine, getting EC2 instance ready

# Further EDA

In [ ]:
# What is the distribution of ratings per restaurant and ratings per user?
ratings_per_restaurant = bus_review_df.groupBy()


In [ ]:
spark.stop()

# Troubleshoot query errors

In [36]:
business_df = pd.read_json('data/business.json', lines = True)

In [37]:
# drop messy/unneeded columns
col_drop = ['attributes', 'hours', 'review_count', 'postal_code']
business_df.drop(columns = col_drop, inplace = True)
# drop messy rows
business_df = business_df[~business_df.categories.isnull()]

In [38]:
# Query down to restaurants in AZ
business_df = business_df[business_df.categories.str.contains('Restaurants')]
business_df = business_df[business_df.state == 'AZ']

In [39]:
business_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11465 entries, 11 to 192532
Data columns (total 10 columns):
address        11465 non-null object
business_id    11465 non-null object
categories     11465 non-null object
city           11465 non-null object
is_open        11465 non-null int64
latitude       11465 non-null float64
longitude      11465 non-null float64
name           11465 non-null object
stars          11465 non-null float64
state          11465 non-null object
dtypes: float64(3), int64(1), object(6)
memory usage: 985.3+ KB


In [ ]:
bus_review_df = pd.merge(sample_df, business_df, 
                         on='business_id', how='left')

# Experiment with chunksize on pandas read_json

In [42]:
reader = pd.read_json('data/review.json', lines = True, chunksize = 100000)

In [48]:
for idx, chunk in enumerate(reader):
    pass

In [49]:
idx

53

# Explore modeling with small subset

In [6]:
sample_df = pd.read_json('data/review_sample.json', lines = True)

In [22]:
sample_df = pd.read_json('data/review_sample.json', lines = True)
sample_df.drop(columns = ['cool', 'date', 'review_id',
                        'funny', 'text', 'useful'], inplace = True)

In [23]:
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
business_id    5000 non-null object
stars          5000 non-null int64
user_id        5000 non-null object
dtypes: int64(1), object(2)
memory usage: 117.3+ KB


In [28]:
y = sample_df['stars']
X = sample_df.drop('stars', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

identifier_df_train = X_train[['user_id', 'business_id']]
identifier_df_test = X_test[['user_id', 'business_id']]

In [ ]:
all_predictions = []
all_ratings = []
# Iterate over all algorithms
for algorithm in [SVD(), surprise.SlopeOne(), surprise.NMF(), surprise.NormalPredictor(), surprise.KNNBaseline(), surprise.KNNBasic(), surprise.KNNWithMeans(), surprise.KNNWithZScore(), surprise.BaselineOnly(), surprise.CoClustering()]:
    # fit the model
    alg_name = str(algorithm)[str(algorithm).find('ization')+8 : str(algorithm).find('obj')-1]
    print('Fitting algorithm {}'.format(alg_name))
    algorithm.fit(train_set)

    # run predictions over
    print('Predicting algorithm {}'.format(alg_name))
    model_predictions = []
    model_ratings = []
    for idx, row in X_test.iterrows():
        prediction = algorithm.predict(row[0], row[1])
        model_predictions.append(prediction)
        model_ratings.append(prediction[3])
    
    pred_pkl_file = alg_name + '_predictions.pkl'
    ratings_pkl_file = alg_name + '_ratings.pkl'
    pickle.dump(model_predictions, open(pred_pkl_file, 'wb'))
    pickle.dump(model_ratings, open(ratings_pkl_file, 'wb'))


In [ ]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)